# Import Labraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

# Connection To Database

In [3]:
conn = sqlite3.connect("Upi_Digital_Payment.db")
cursor = conn.cursor()


# Load Dataset


In [4]:
df = pd.read_csv("upi_digital_payment_advanced.csv")
df.to_sql("Digital_Payment",conn,if_exists="append",index=False)

5000

# Dataset Features

In [24]:
print("Size of Dataframe:\n\n",df.size)
print("Shape of Dataframe:\n\n",df.shape)
print("info of Dataframe:\n\n",df.info())
print("Description of Dataframe:\n\n",df.describe())
print(df.columns)

Size of Dataframe:

 50000
Shape of Dataframe:

 (5000, 10)
<class 'pandas.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    5000 non-null   str           
 1   user_id           5000 non-null   str           
 2   city              5000 non-null   str           
 3   transaction_date  5000 non-null   datetime64[us]
 4   amount            5000 non-null   float64       
 5   payment_app       5000 non-null   str           
 6   transaction_type  5000 non-null   str           
 7   device_type       5000 non-null   str           
 8   is_fraud          5000 non-null   int64         
 9   Month             5000 non-null   str           
dtypes: datetime64[us](1), float64(1), int64(1), str(7)
memory usage: 390.8 KB
info of Dataframe:

 None
Description of Dataframe:

                  transaction_date        amount     is_fraud
co

# Dataset Cleaning

In [8]:
print("Null Values From Dataset:\n",df.isna().sum())
print("Duplicate Values From Dateset:",df.duplicated().sum())


Null Values From Dataset:
 transaction_id      0
user_id             0
city                0
transaction_date    0
amount              0
payment_app         0
transaction_type    0
device_type         0
is_fraud            0
dtype: int64
Duplicate Values From Dateset: 0


In [23]:
# Convert Date Column into Datetime

df["transaction_date"] = pd.to_datetime(df["transaction_date"])
df["Month"] = df["transaction_date"].dt.month_name()
print(df[["transaction_date"]])
print(df[["Month"]])

        transaction_date
0    2024-12-14 21:20:20
1    2024-03-28 12:15:31
2    2024-12-09 17:29:15
3    2024-10-03 23:05:04
4    2024-02-28 23:27:58
...                  ...
4995 2024-08-28 04:21:40
4996 2024-06-08 16:34:18
4997 2024-01-20 15:09:58
4998 2024-03-04 00:15:44
4999 2024-06-15 19:22:00

[5000 rows x 1 columns]
         Month
0     December
1        March
2     December
3      October
4     February
...        ...
4995    August
4996      June
4997   January
4998     March
4999      June

[5000 rows x 1 columns]


# Check Dataload In Created Table

In [10]:
query = "PRAGMA table_info(Digital_Payment)"
info = pd.read_sql(query,conn)
print(info)

   cid              name     type  notnull dflt_value  pk
0    0    transaction_id     TEXT        0       None   0
1    1           user_id     TEXT        0       None   0
2    2              city     TEXT        0       None   0
3    3  transaction_date     TEXT        0       None   0
4    4            amount     REAL        0       None   0
5    5       payment_app     TEXT        0       None   0
6    6  transaction_type     TEXT        0       None   0
7    7       device_type     TEXT        0       None   0
8    8          is_fraud  INTEGER        0       None   0


# Fraud Via Transaction Type

In [12]:
query = """SELECT transaction_type,user_id,city,
payment_app,device_type,AVG(is_fraud)
FROM Digital_Payment
GROUP BY transaction_type,user_id,city,
payment_app,device_type
ORDER BY AVG(is_fraud) DESC
"""

top_transactiontype_fraud = pd.read_sql(query,conn)
print(top_transactiontype_fraud)


     transaction_type   user_id       city payment_app device_type  \
0        Bill Payment  USER1258    Lucknow       Paytm     Android   
1        Bill Payment  USER1480    Kolkata  Amazon Pay     Android   
2        Bill Payment  USER1492    Kolkata     PhonePe         iOS   
3        Bill Payment  USER1561       Pune       Paytm         iOS   
4        Bill Payment  USER1711  Ahmedabad     PhonePe         Web   
...               ...       ...        ...         ...         ...   
4994         Recharge  USER9970  Bangalore       Paytm         iOS   
4995         Recharge  USER9973  Bangalore  Amazon Pay     Android   
4996         Recharge  USER9976  Bangalore       Paytm         Web   
4997         Recharge  USER9988     Mumbai       Paytm     Android   
4998         Recharge  USER9992  Bangalore  Amazon Pay         iOS   

      AVG(is_fraud)  
0               1.0  
1               1.0  
2               1.0  
3               1.0  
4               1.0  
...             ...  
4994 